In [1]:
!pip install datasets
from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, DatasetDict
import os
import shutil
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
train_csv_file = '/content/drive/MyDrive/Handwrittenprescription/Training/training_labels.csv'
train_img_file = '/content/drive/MyDrive/Handwrittenprescription/Training/training_words'
train_output_dir = '/content/drive/MyDrive/edited_trainset'
test_csv_file = '/content/drive/MyDrive/Handwrittenprescription/Testing/testing_labels.csv'
test_img_file = '/content/drive/MyDrive/Handwrittenprescription/Testing/testing_words'
test_output_dir = '/content/drive/MyDrive/edited_testset'

In [ ]:
dataset = load_dataset('imagefolder',data_dir = train_output_dir)
print(dataset)
print(dataset['train'][0])
dataseet = load_dataset('imagefolder',data_dir = test_output_dir)
if 'validation' not in dataset:
    dataset['validation'] = dataseet['train']

Resolving data files:   0%|          | 0/3120 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 3120
    })
})
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=132x81 at 0x7DF36F2EE290>, 'label': 0}


Resolving data files:   0%|          | 0/780 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    images = [image.convert("RGB") for image in examples['image']]
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    labels = [processor.tokenizer(str(text), add_special_tokens=True).input_ids for text in examples['label']]
    return {"pixel_values": pixel_values, "labels": labels}

train_dataset = dataset['train'].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

Map:   0%|          | 0/3120 [00:00<?, ? examples/s]

In [ ]:
eval_dataset = dataset['validation'].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['validation'].column_names
)

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

Step,Training Loss
500,1.924300
1000,2.007800
1500,1.796300
2000,1.582400


TrainOutput(global_step=2340, training_loss=1.7685953254373665, metrics={'train_runtime': 4476.8991, 'train_samples_per_second': 2.091, 'train_steps_per_second': 0.523, 'total_flos': 7.00394938772226e+18, 'train_loss': 1.7685953254373665, 'epoch': 3.0})

In [4]:
model.save_pretrained('/content/drive/MyDrive/fine_tuned_model')
processor.save_pretrained('/content/drive/MyDrive/fine_tuned_model')

NameError: name 'model' is not defined

In [32]:
model = VisionEncoderDecoderModel.from_pretrained('/content/drive/MyDrive/fine_tuned_model')

processor = TrOCRProcessor.from_pretrained('/content/drive/MyDrive/fine_tuned_model')

In [2]:
model = VisionEncoderDecoderModel.from_pretrained('/content/drive/MyDrive/fine_tuned_model')
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')
processor = TrOCRProcessor.from_pretrained('/content/drive/MyDrive/fine_tuned_model')

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
image_url = '/content/drive/MyDrive/PS2-Samples-HackRX5/Sample14.png'
image = Image.open(image_url).convert("RGB")

In [8]:
pixel_values = processor(images=image, return_tensors="pt").pixel_values
print(pixel_values)
generated_ids = model.generate(pixel_values)
print(generated_ids)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]])
tensor([[    2,   134,   112,    20, 10797, 19616,  1535,     2]])


In [36]:
opixel_values = processor(images=image, return_tensors="pt").pixel_values
print(opixel_values)
ogenerated_ids = model.generate(opixel_values)
print(ogenerated_ids)
ogenerated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]])
tensor([[    2,   134,   112,    20, 10797, 19616,  1535,     2]])


In [29]:
final_text = "\n".join(generated_text)
print(final_text)

1
 
1
 
T
h
e
 
T
e
n
n
e
s
s
e
r
e
s
1 1 The Tennesseres
